In [ ]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import numpy as np
import matplotlib.pyplot as plt
import math
from matplotlib.animation import FuncAnimation

def Harmonic_Model():
    input = Input(shape=(1,))
    x = Dense(100, activation="tanh")(input)
    x = Dense(200, activation="tanh")(x)
    x = Dense(200, activation="tanh")(x)
    x = Dense(400, activation="tanh")(x)
    x = Dense(400, activation="tanh")(x)
    x = Dense(400, activation="tanh")(x)
    out = Dense(1, activation=None)(x)
    model = Model(input, out)
    return model


model = Harmonic_Model() #Model creation

x = np.linspace(0, 2, 1000)
x_tf = tf.convert_to_tensor(x[:, np.newaxis], dtype=tf.float32)
# Initial condition
t_0 = tf.convert_to_tensor([[0.0]], dtype=tf.float32)  
y_0 = tf.convert_to_tensor([[1.0]], dtype=tf.float32) 
y_prime_0 = tf.convert_to_tensor([[0.0]], dtype=tf.float32)
y = np.exp(-2 * x) * (np.cos(6 * math.sqrt(11) * x) + (math.sqrt(11) / 33) * np.sin(6 * math.sqrt(11) * x))
t_0_value = 0
y_0_value = 1
fig, ax = plt.subplots()
line, = ax.plot([],[], lw=3 , color='red', label='Predictions')
ax.set_xlabel("Time")
ax.set_ylabel("Displacement")
ax.set_xlim(0,2)
ax.set_ylim(-1.1,1.2)
ax.plot(x, y, '--', label="True", color="green")
ax.scatter([t_0_value], [y_0_value], color="blue", label="Initial Condition", zorder=6)
ax.legend()
def init():
    line.set_data([],[])
    return line,


def loss_function(x):
    with tf.GradientTape() as tape:
        tape.watch(t_0)
        model_pred0 = model(t_0)
    grad0 = tape.gradient(model_pred0, t_0)
    with tf.GradientTape() as tape2:
        tape2.watch(x)
        with tf.GradientTape() as tape1:
            tape1.watch(x)
            model_pred = model(x)
        grad1 = tape1.gradient(model_pred, x)
    grad2 = tape2.gradient(grad1, x)
    initial_condition_loss = tf.reduce_mean(tf.square((model_pred0 - y_0))) + tf.reduce_mean(tf.square(((grad0 - y_prime_0))))

    physics_loss = tf.reduce_mean(tf.square((grad2 + 4*grad1 + 400*model_pred)))

    total_loss = initial_condition_loss + physics_loss
    return total_loss

optimizer = tf.keras.optimizers.Adam(learning_rate = 0.001)
def update(epoch):
    for i in range(5):
        with tf.GradientTape() as final_tape:
            loss = loss_function(x_tf)
        changes = final_tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(changes, model.trainable_variables))

    # Output for every 10th epoch
    if (epoch + 1) % 10 == 0:
        print(f"Epoch {(epoch + 1)}, Loss: {loss.numpy()}")

    # predictions for plotting
    t_values_tf = tf.convert_to_tensor(x[:, np.newaxis], dtype=tf.float32)
    predictions = model(t_values_tf).numpy()

    line.set_data(x, predictions)
    ax.set_title(f"Harmonic Oscillation Prediction using PINN\nEpoch {(epoch + 1)}, Loss: {loss.numpy():.4f}")
    return line,

# Create the animation
ani = FuncAnimation(fig, update, frames=3000, init_func=init, blit=True)
ani.save('training_animation3.gif', writer='pillow', fps=6)
